StateInven_vs_past3YrsNPP_DummyEW_Dec11-for-Dec13

cow_inventory as function of moving average of past 3 years of NPP.

DummyEW: Dummy variable for east vs. western states.

On Dec. 11 Mike and I had a meeting and this decision was made.

In [1]:
import shutup

shutup.please()

import pandas as pd
import numpy as np
from datetime import datetime
import os, os.path, pickle, sys

from sklearn import preprocessing
import statistics
import statsmodels.api as sm

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

sys.path.append("/Users/hn/Documents/00_GitHub/Rangeland/Python_Codes/")
import rangeland_core as rc

In [2]:
data_dir_base = "/Users/hn/Documents/01_research_data/RangeLand/Data/"
census_population_dir = data_dir_base + "census/"
# Shannon_data_dir = data_dir_base + "Shannon_Data/"
# USDA_data_dir = data_dir_base + "/NASS_downloads/"
param_dir = data_dir_base + "parameters/"
Min_data_base = data_dir_base + "Min_Data/"
reOrganized_dir = data_dir_base + "reOrganized/"

In [3]:
# for bold print
start_b = "\033[1m"
end_b = "\033[0;0m"
print ("This is " + start_b + "a_bold_text" + end_b + "!")

This is a_bold_text!


# Read

In [4]:
SoI = [
    "Alabama",
    "Arizona",
    "Arkansas",
    "California",
    "Colorado",
    "Florida",
    "Georgia",
    "Idaho",
    "Illinois",
    "Iowa",
    "Kansas",
    "Kentucky",
    "Louisiana",
    "Mississippi",
    "Missouri",
    "Montana",
    "Nebraska",
    "Nevada",
    "New Mexico",
    "North Dakota",
    "Oklahoma",
    "Oregon",
    "South Dakota",
    "Tennessee",
    "Texas",
    "Utah",
    "Virginia",
    "Washington",
    "Wyoming",
]

abb_dict = pd.read_pickle(param_dir + "state_abbreviations.sav")
SoI_abb = []
for x in SoI:
    SoI_abb = SoI_abb + [abb_dict["full_2_abb"][x]]

In [5]:
county_fips = pd.read_pickle(reOrganized_dir + "county_fips.sav")
county_fips = county_fips["county_fips"]
county_fips = county_fips[["state", "state_fip", "EW"]]
county_fips = county_fips[county_fips.state.isin(SoI_abb)].copy()
county_fips.drop_duplicates(inplace=True)
county_fips.reset_index(drop=True, inplace=True)

county_fips.head(2)

,state,state_fip,EW
0,AL,01,E
1,AR,05,E


In [6]:
state_SoI_fip = county_fips.state_fip.unique()
len(state_SoI_fip)

29

In [7]:
# herb = pd.read_csv(data_dir_base + "Supriya/Nov30_herb/state_herb_ratio.csv")
# herb = rc.correct_state_int_fips_to_str(df=herb, col_="state_fip")
# herb.sort_values(by=["state_fip"], inplace=True)
herb = pd.read_pickle(data_dir_base + "Supriya/Nov30_HerbRatio/state_herb_ratio.sav")
print (herb.keys())
herb = herb["state_herb_ratio"]
herb = herb[herb.state_fip.isin(state_SoI_fip)]
# herb.dropna(how="any", inplace=True)
herb.head(3)

dict_keys(['state_herb_ratio', 'source_code', 'Author', 'Date'])


,state_fip,ag_land,pixel_count,herb_avg,herb_std
0,01,131174048583,10874.0,21.019864,25.356867
2,04,294198551143,3704408.0,45.168734,17.311673
3,05,134768872727,24951.0,27.282554,29.379988


In [8]:
herb.dropna(how="any", inplace=True)
herb.reset_index(drop=True, inplace=True)
herb.head(3)

,state_fip,ag_land,pixel_count,herb_avg,herb_std
0,01,131174048583,10874.0,21.019864,25.356867
1,04,294198551143,3704408.0,45.168734,17.311673
2,05,134768872727,24951.0,27.282554,29.379988


### Read Rangeland area and Total area:

In [9]:
# Rangeland area and Total area:
state_RA = pd.read_pickle(reOrganized_dir + "state_RA_area.sav")
state_RA = state_RA["state_RA_area"]
state_RA = state_RA[state_RA.state_fip.isin(state_SoI_fip)]
state_RA.head(2)

,state_fip,rangeland_acre,state_area_acre
0,01,31570.99,14825637.76
1,04,54476918.89,72998287.36


### Read NPP

In [10]:
NPP = pd.read_csv(Min_data_base + "statefips_annual_MODIS_NPP.csv")
NPP.rename(columns={"NPP": "modis_npp", "statefips90m": "state_fip"}, inplace=True)
NPP = rc.correct_3digitStateFips_Min(NPP, "state_fip")

NPP = NPP[NPP.state_fip.isin(state_SoI_fip)]
NPP.reset_index(drop=True, inplace=True)

NPP.head(2)

,year,state_fip,modis_npp
0,2001,01,0.816597
1,2001,04,0.115683


### Convert Unit NPP to State Level

In [13]:
state_NPP_Ra = pd.merge(NPP, state_RA, on=["state_fip"], how="left")

state_NPP_Ra = rc.covert_unitNPP_2_total(NPP_df=state_NPP_Ra, 
                                         npp_unit_col_="modis_npp",
                                         acr_area_col_="rangeland_acre", 
                                         npp_area_col_="state_rangeland_npp")

### Security check to not make mistake later:
state_NPP_Ra.drop(columns=["modis_npp"], inplace=True)
state_NPP_Ra.head(2)

,year,state_fip,rangeland_acre,state_area_acre,area_m2,state_rangeland_npp
0,2001,01,31570.99,14825637.76,1.277635e+08,1.043312e+08
1,2001,04,54476918.89,72998287.36,2.204606e+11,2.550357e+10


### Read Inventory

In [14]:
invent_tall = pd.read_pickle(reOrganized_dir + "Shannon_Beef_Cows_fromCATINV_tall.sav")
invent_tall = invent_tall["CATINV_annual_tall"]

invent_tall = invent_tall[invent_tall.state.isin(SoI_abb)]

invent_tall = invent_tall[invent_tall.year.isin(state_NPP_Ra.year.unique())]
invent_tall.reset_index(drop=True, inplace=True)
invent_tall.head(2)

,state,year,inventory,state_fip
0,AL,2001,737000.0,01
1,AL,2002,750000.0,01


In [15]:
col_3NPP = ["NPP_3", "NPP_2", "NPP_1"]

invent_tall[col_3NPP] = 0
invent_tall.head(2)

,state,year,inventory,state_fip,NPP_3,NPP_2,NPP_1
0,AL,2001,737000.0,01,0,0,0
1,AL,2002,750000.0,01,0,0,0


In [16]:
for a_state in invent_tall.state_fip.unique():
    for a_year in np.arange(invent_tall.year.min()+3, invent_tall.year.max()+1):
        past_3_yrs = [a_year-3, a_year-2, a_year-1]
        curr_NPP = state_NPP_Ra[(state_NPP_Ra.year.isin(past_3_yrs)) & (state_NPP_Ra.state_fip == a_state)]

        NPP3yrs = curr_NPP.state_rangeland_npp.values # list(curr_NPP.state_rangeland_npp)
        invent_tall.loc[(invent_tall.year == a_year) & (invent_tall.state_fip == a_state), col_3NPP] = NPP3yrs

In [17]:
invent_tall.head(5)

,state,year,inventory,state_fip,NPP_3,NPP_2,NPP_1
0,AL,2001,737000.0,01,0.000000e+00,0.000000e+00,0.000000e+00
1,AL,2002,750000.0,01,0.000000e+00,0.000000e+00,0.000000e+00
2,AL,2003,761000.0,01,0.000000e+00,0.000000e+00,0.000000e+00
3,AL,2004,732000.0,01,1.043312e+08,8.962134e+07,1.110775e+08
4,AL,2005,724000.0,01,8.962134e+07,1.110775e+08,1.017458e+08


In [18]:
invent_tall.tail(5)

,state,year,inventory,state_fip,NPP_3,NPP_2,NPP_1
575,WY,2016,704000.0,56,3.097898e+10,4.041571e+10,4.230707e+10
576,WY,2017,714000.0,56,4.041571e+10,4.230707e+10,3.746571e+10
577,WY,2018,714000.0,56,4.230707e+10,3.746571e+10,3.961930e+10
578,WY,2019,714000.0,56,3.746571e+10,3.961930e+10,3.896260e+10
579,WY,2020,724000.0,56,3.961930e+10,3.896260e+10,4.418175e+10


## First three years
can be either dropped or drop the first year, and for second year use only 1 year of NPP, and for third year use only 2 years of NPP.

Let's just drop them.

In [19]:
invent_tall = invent_tall[invent_tall.year>=2004].copy()

invent_tall.reset_index(drop=True, inplace=True)
invent_tall.head(2)

,state,year,inventory,state_fip,NPP_3,NPP_2,NPP_1
0,AL,2004,732000.0,01,1.043312e+08,8.962134e+07,1.110775e+08
1,AL,2005,724000.0,01,8.962134e+07,1.110775e+08,1.017458e+08


In [20]:
state_RA.head(2)

,state_fip,rangeland_acre,state_area_acre
0,01,31570.99,14825637.76
1,04,54476918.89,72998287.36


In [21]:
invent_tall = pd.merge(invent_tall, state_RA[["state_fip", "rangeland_acre"]], 
                       on=["state_fip"], how="left")
invent_tall.head(2)

,state,year,inventory,state_fip,NPP_3,NPP_2,NPP_1,rangeland_acre
0,AL,2004,732000.0,01,1.043312e+08,8.962134e+07,1.110775e+08,31570.99
1,AL,2005,724000.0,01,8.962134e+07,1.110775e+08,1.017458e+08,31570.99


In [22]:
herb.head(2)

,state_fip,ag_land,pixel_count,herb_avg,herb_std
0,01,131174048583,10874.0,21.019864,25.356867
1,04,294198551143,3704408.0,45.168734,17.311673


In [23]:
invent_tall = pd.merge(invent_tall, herb[["state_fip", "herb_avg"]], 
                       on=["state_fip"], how="left")
invent_tall.head(2)

,state,year,inventory,state_fip,NPP_3,NPP_2,NPP_1,rangeland_acre,herb_avg
0,AL,2004,732000.0,01,1.043312e+08,8.962134e+07,1.110775e+08,31570.99,21.019864
1,AL,2005,724000.0,01,8.962134e+07,1.110775e+08,1.017458e+08,31570.99,21.019864


In [24]:
county_fips.head(2)

,state,state_fip,EW
0,AL,01,E
1,AR,05,E


In [25]:
invent_tall = pd.merge(invent_tall, county_fips[["state_fip", "EW"]], 
                       on=["state_fip"], how="left")
invent_tall.head(2)

,state,year,inventory,state_fip,NPP_3,NPP_2,NPP_1,rangeland_acre,herb_avg,EW
0,AL,2004,732000.0,01,1.043312e+08,8.962134e+07,1.110775e+08,31570.99,21.019864,E
1,AL,2005,724000.0,01,8.962134e+07,1.110775e+08,1.017458e+08,31570.99,21.019864,E


In [26]:
invent_tall["EW_binary"] = invent_tall["EW"].map({"E":0, "W" : 1})
invent_tall.head(2)

,state,year,inventory,state_fip,NPP_3,NPP_2,NPP_1,rangeland_acre,herb_avg,EW,EW_binary
0,AL,2004,732000.0,01,1.043312e+08,8.962134e+07,1.110775e+08,31570.99,21.019864,E,0
1,AL,2005,724000.0,01,8.962134e+07,1.110775e+08,1.017458e+08,31570.99,21.019864,E,0


In [27]:
# Re-order the columns.
new_order = ["state_fip", "year", "NPP_3", "NPP_2", "NPP_1", 
             "rangeland_acre", "herb_avg", "EW", "EW_binary", "inventory"]
invent_tall = invent_tall[new_order]
invent_tall.head(2)

,state_fip,year,NPP_3,NPP_2,NPP_1,rangeland_acre,herb_avg,EW,EW_binary,inventory
0,01,2004,1.043312e+08,8.962134e+07,1.110775e+08,31570.99,21.019864,E,0,732000.0
1,01,2005,8.962134e+07,1.110775e+08,1.017458e+08,31570.99,21.019864,E,0,724000.0


In [28]:
invent_tall.tail(2)

,state_fip,year,NPP_3,NPP_2,NPP_1,rangeland_acre,herb_avg,EW,EW_binary,inventory
491,56,2019,3.746571e+10,3.961930e+10,3.896260e+10,46079514.69,68.307845,W,1,714000.0
492,56,2020,3.961930e+10,3.896260e+10,4.418175e+10,46079514.69,68.307845,W,1,724000.0


## Model inventory vs. 3 yrs NPP.

In [29]:
indp_vars = col_3NPP
y_var = "inventory"

In [30]:
print(invent_tall.year.unique().min())
yr_max = invent_tall.year.unique().max()
print(yr_max)

2004
2020


In [31]:
train_df = invent_tall[invent_tall.year < yr_max].copy()
test_df  = invent_tall[invent_tall.year == yr_max].copy()

In [32]:
X = train_df[indp_vars]
X = sm.add_constant(X)
Y = train_df[y_var].astype(float)
ks = sm.OLS(Y, X)
ks_result =ks.fit()
ks_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              inventory   R-squared:                       0.383
Model:                            OLS   Adj. R-squared:                  0.379
Method:                 Least Squares   F-statistic:                     95.31
Date:                Wed, 03 Jan 2024   Prob (F-statistic):           5.51e-48
Time:                        12:54:14   Log-Likelihood:                -6900.2
No. Observations:                 464   AIC:                         1.381e+04
Df Residuals:                     460   BIC:                         1.382e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.031e+05   4.35e+04     11.567      0.000    4.18e+05    5.89e+05
NPP_3       1.018e-05   5.76e-06      1.767      0.078   -1.14e-06    2.15e-05
NPP_2       7.839e-06   6.01e-06      1.304      0.193   -3.97e-06    1.97e-05
NPP_1       5.626e-06   5.62e-06      1.001      0.318   -5.42e-06    1.67e-05
==============================================================================
Omnibus:                       27.770   Durbin-Watson:                   0.137
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               31.451
Skew:                           0.584   Prob(JB):                     1.48e-07
Kurtosis:                       3.512   Cond. No.                     7.23e+10
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.23e+10. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [33]:
test_A = test_df[indp_vars]
test_A = sm.add_constant(test_A)
y_test = test_df[y_var].astype(float)

yhat_test = test_A @ ks_result.params

NPP_test_res = y_test - yhat_test
NPP_RSS_test = np.dot(NPP_test_res, NPP_test_res)
NPP_MSE_test = NPP_RSS_test / len(y_test)
NPP_RSE_test = np.sqrt(NPP_MSE_test)

print (start_b + "Test residuals for 3-years-NPP:\n" + end_b)
print("    RSS_test = {0:.0f}.".format(NPP_RSS_test))
print("    MSE_test = {0:.0f}.".format(NPP_MSE_test))
print("    RSE =  {0:.0f}.".format(NPP_RSE_test))

Test residuals for 3-years-NPP:

    RSS_test = 13624774153306.
    MSE_test = 469819798390.
    RSE =  685434.


## Model inventory vs. 3 yrs NPP, RA, HerbRatio.

In [34]:
indp_vars = col_3NPP + ["rangeland_acre", "herb_avg"]
y_var = "inventory"
indp_vars

['NPP_3', 'NPP_2', 'NPP_1', 'rangeland_acre', 'herb_avg']

In [35]:
train_df.head(2)

,state_fip,year,NPP_3,NPP_2,NPP_1,rangeland_acre,herb_avg,EW,EW_binary,inventory
0,01,2004,1.043312e+08,8.962134e+07,1.110775e+08,31570.99,21.019864,E,0,732000.0
1,01,2005,8.962134e+07,1.110775e+08,1.017458e+08,31570.99,21.019864,E,0,724000.0


In [36]:
X = train_df[indp_vars]
X = sm.add_constant(X)
Y = train_df[y_var].astype(float)
ks = sm.OLS(Y, X)
ks_result =ks.fit()
ks_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              inventory   R-squared:                       0.578
Model:                            OLS   Adj. R-squared:                  0.574
Method:                 Least Squares   F-statistic:                     125.6
Date:                Wed, 03 Jan 2024   Prob (F-statistic):           1.58e-83
Time:                        12:54:18   Log-Likelihood:                -6812.0
No. Observations:                 464   AIC:                         1.364e+04
Df Residuals:                     458   BIC:                         1.366e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           8.184e+05    7.1e+04     11.522      0.000    6.79e+05    9.58e+05
NPP_3           2.141e-05   4.84e-06      4.423      0.000    1.19e-05    3.09e-05
NPP_2           1.854e-05   5.04e-06      3.681      0.000    8.64e-06    2.84e-05
NPP_1           2.104e-05   4.79e-06      4.396      0.000    1.16e-05    3.04e-05
rangeland_acre    -0.0394      0.003    -14.507      0.000      -0.045      -0.034
herb_avg       -4857.6312   1335.089     -3.638      0.000   -7481.291   -2233.972
==============================================================================
Omnibus:                       35.835   Durbin-Watson:                   0.173
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              147.924
Skew:                           0.110   Prob(JB):                     7.56e-33
Kurtosis:                       5.757   Cond. No.                     1.43e+11
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.43e+11. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [37]:
test_A = test_df[indp_vars]
test_A = sm.add_constant(test_A)
y_test = test_df[y_var].astype(float)

yhat_test = test_A @ ks_result.params

NPP_test_res = y_test - yhat_test
NPP_RSS_test = np.dot(NPP_test_res, NPP_test_res)
NPP_MSE_test = NPP_RSS_test / len(y_test)
NPP_RSE_test = np.sqrt(NPP_MSE_test)

print (start_b + "Test residuals for 3-years-NPP and herb ratio and rangeland area:\n" + end_b)
print("    RSS_test = {0:.0f}.".format(NPP_RSS_test))
print("    MSE_test = {0:.0f}.".format(NPP_MSE_test))
print("    RSE =  {0:.0f}.".format(NPP_RSE_test))

Test residuals for 3-years-NPP and herb ratio and rangeland area:

    RSS_test = 9815961697101.
    MSE_test = 338481437831.
    RSE =  581792.


## Model inventory vs. 3 yrs NPP and EW Dummy.

In [38]:
invent_tall.head(2)

,state_fip,year,NPP_3,NPP_2,NPP_1,rangeland_acre,herb_avg,EW,EW_binary,inventory
0,01,2004,1.043312e+08,8.962134e+07,1.110775e+08,31570.99,21.019864,E,0,732000.0
1,01,2005,8.962134e+07,1.110775e+08,1.017458e+08,31570.99,21.019864,E,0,724000.0


In [39]:
indp_vars = col_3NPP + ["EW_binary"]
indp_vars

['NPP_3', 'NPP_2', 'NPP_1', 'EW_binary']

In [40]:
train_df.head(2)

,state_fip,year,NPP_3,NPP_2,NPP_1,rangeland_acre,herb_avg,EW,EW_binary,inventory
0,01,2004,1.043312e+08,8.962134e+07,1.110775e+08,31570.99,21.019864,E,0,732000.0
1,01,2005,8.962134e+07,1.110775e+08,1.017458e+08,31570.99,21.019864,E,0,724000.0


In [41]:
X = train_df[indp_vars]
X = sm.add_constant(X)
Y = train_df[y_var].astype(float)
ks = sm.OLS(Y, X)
ks_result =ks.fit()
ks_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              inventory   R-squared:                       0.545
Model:                            OLS   Adj. R-squared:                  0.541
Method:                 Least Squares   F-statistic:                     137.6
Date:                Wed, 03 Jan 2024   Prob (F-statistic):           3.73e-77
Time:                        12:54:29   Log-Likelihood:                -6829.5
No. Observations:                 464   AIC:                         1.367e+04
Df Residuals:                     459   BIC:                         1.369e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       7.843e+05   4.34e+04     18.078      0.000    6.99e+05     8.7e+05
NPP_3       1.428e-05   4.97e-06      2.875      0.004    4.52e-06     2.4e-05
NPP_2       1.177e-05   5.18e-06      2.274      0.023     1.6e-06    2.19e-05
NPP_1       1.272e-05   4.87e-06      2.615      0.009    3.16e-06    2.23e-05
EW_binary  -1.016e+06   7.95e+04    -12.782      0.000   -1.17e+06    -8.6e+05
==============================================================================
Omnibus:                       16.011   Durbin-Watson:                   0.143
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               17.012
Skew:                           0.418   Prob(JB):                     0.000202
Kurtosis:                       3.425   Cond. No.                     1.61e+11
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.61e+11. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [42]:
test_A = test_df[indp_vars]
test_A = sm.add_constant(test_A)
y_test = test_df[y_var].astype(float)

yhat_test = test_A @ ks_result.params

NPP_test_res = y_test - yhat_test
NPP_RSS_test = np.dot(NPP_test_res, NPP_test_res)
NPP_MSE_test = NPP_RSS_test / len(y_test)
NPP_RSE_test = np.sqrt(NPP_MSE_test)

print (start_b + "Test residuals for 3-years-NPP and dummy east-west:\n" + end_b)
print("    RSS_test = {0:.0f}.".format(NPP_RSS_test))
print("    MSE_test = {0:.0f}.".format(NPP_MSE_test))
print("    RSE =  {0:.0f}.".format(NPP_RSE_test))

Test residuals for 3-years-NPP and dummy east-west:

    RSS_test = 10384607336039.
    MSE_test = 358089908139.
    RSE =  598406.


## Model inventory vs. 3 yrs NPP, RA, HerbRatio, and EW Dummy.

In [43]:
invent_tall.head(2)

,state_fip,year,NPP_3,NPP_2,NPP_1,rangeland_acre,herb_avg,EW,EW_binary,inventory
0,01,2004,1.043312e+08,8.962134e+07,1.110775e+08,31570.99,21.019864,E,0,732000.0
1,01,2005,8.962134e+07,1.110775e+08,1.017458e+08,31570.99,21.019864,E,0,724000.0


In [44]:
indp_vars = col_3NPP + ["rangeland_acre", "herb_avg", "EW_binary"]
indp_vars

['NPP_3', 'NPP_2', 'NPP_1', 'rangeland_acre', 'herb_avg', 'EW_binary']

In [45]:
X = sm.add_constant(train_df[indp_vars])
Y = train_df[y_var].astype(float)
ks = sm.OLS(Y, X)
ks_result =ks.fit()
ks_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              inventory   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.649
Method:                 Least Squares   F-statistic:                     143.6
Date:                Wed, 03 Jan 2024   Prob (F-statistic):          8.08e-102
Time:                        12:54:45   Log-Likelihood:                -6766.5
No. Observations:                 464   AIC:                         1.355e+04
Df Residuals:                     457   BIC:                         1.358e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const            4.47e+05   7.45e+04      6.001      0.000    3.01e+05    5.93e+05
NPP_3           1.912e-05    4.4e-06      4.347      0.000    1.05e-05    2.78e-05
NPP_2           1.643e-05   4.58e-06      3.590      0.000    7.44e-06    2.54e-05
NPP_1           1.792e-05   4.36e-06      4.115      0.000    9.36e-06    2.65e-05
rangeland_acre    -0.0190      0.003     -5.941      0.000      -0.025      -0.013
herb_avg        9517.4515   1885.246      5.048      0.000    5812.626    1.32e+04
EW_binary      -1.162e+06   1.17e+05     -9.953      0.000   -1.39e+06   -9.33e+05
==============================================================================
Omnibus:                       25.000   Durbin-Watson:                   0.168
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               57.358
Skew:                           0.255   Prob(JB):                     3.51e-13
Kurtosis:                       4.645   Cond. No.                     2.76e+11
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.76e+11. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [46]:
test_A = test_df[indp_vars]
test_A = sm.add_constant(test_A)
y_test = test_df[y_var].astype(float)

yhat_test = test_A @ ks_result.params

NPP_test_res = y_test - yhat_test
NPP_RSS_test = np.dot(NPP_test_res, NPP_test_res)
NPP_MSE_test = NPP_RSS_test / len(y_test)
NPP_RSE_test = np.sqrt(NPP_MSE_test)

print (start_b + "Test residuals for 3-years-NPP, RA, herb ratio, and dummy east-west:\n" + end_b)
print("    RSS_test = {0:.0f}.".format(NPP_RSS_test))
print("    MSE_test = {0:.0f}.".format(NPP_MSE_test))
print("    RSE =  {0:.0f}.".format(NPP_RSE_test))

Test residuals for 3-years-NPP, RA, herb ratio, and dummy east-west:

    RSS_test = 7771519259855.
    MSE_test = 267983422754.
    RSE =  517671.


### Normalize and model with $\ln(y)$

In [47]:
invent_tall.head(2)

,state_fip,year,NPP_3,NPP_2,NPP_1,rangeland_acre,herb_avg,EW,EW_binary,inventory
0,01,2004,1.043312e+08,8.962134e+07,1.110775e+08,31570.99,21.019864,E,0,732000.0
1,01,2005,8.962134e+07,1.110775e+08,1.017458e+08,31570.99,21.019864,E,0,724000.0


In [48]:
all_indp_vars = ["NPP_3", "NPP_2", "NPP_1", "rangeland_acre", "herb_avg"]
all_indp_vars = sorted(all_indp_vars)
all_indp_vars

['NPP_1', 'NPP_2', 'NPP_3', 'herb_avg', 'rangeland_acre']

In [50]:
# standard_indp = preprocessing.scale(all_df[explain_vars_herb]) # this is biased
normal_df = (invent_tall[all_indp_vars] - invent_tall[all_indp_vars].mean()) / \
                         invent_tall[all_indp_vars].std(ddof=1)
normal_df.head(2)

,NPP_1,NPP_2,NPP_3,herb_avg,rangeland_acre
0,-0.883372,-0.878229,-0.874809,-1.440068,-0.910685
1,-0.883762,-0.877315,-0.875436,-1.440068,-0.910685


In [51]:
normal_cols = [i + j for i, j in zip(all_indp_vars, ["_normal"] * len(all_indp_vars))]
normal_cols

['NPP_1_normal',
 'NPP_2_normal',
 'NPP_3_normal',
 'herb_avg_normal',
 'rangeland_acre_normal']

In [52]:
invent_tall[normal_cols] = normal_df
invent_tall.head(2)

,state_fip,year,NPP_3,NPP_2,NPP_1,rangeland_acre,herb_avg,EW,EW_binary,inventory,NPP_1_normal,NPP_2_normal,NPP_3_normal,herb_avg_normal,rangeland_acre_normal
0,01,2004,1.043312e+08,8.962134e+07,1.110775e+08,31570.99,21.019864,E,0,732000.0,-0.883372,-0.878229,-0.874809,-1.440068,-0.910685
1,01,2005,8.962134e+07,1.110775e+08,1.017458e+08,31570.99,21.019864,E,0,724000.0,-0.883762,-0.877315,-0.875436,-1.440068,-0.910685


In [54]:
X_normal = invent_tall[normal_cols]
X_normal = sm.add_constant(X_normal)
Y = np.log(invent_tall[y_var].astype(float))
ks_normal = sm.OLS(Y, X_normal)
ks_normal_result =ks_normal.fit()
ks_normal_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              inventory   R-squared:                       0.509
Model:                            OLS   Adj. R-squared:                  0.504
Method:                 Least Squares   F-statistic:                     101.0
Date:                Wed, 03 Jan 2024   Prob (F-statistic):           6.05e-73
Time:                        12:59:55   Log-Likelihood:                -356.85
No. Observations:                 493   AIC:                             725.7
Df Residuals:                     487   BIC:                             750.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    13.5380      0.023    598.690      0.000      13.494      13.582
NPP_1_normal              0.3665      0.094      3.912      0.000       0.182       0.551
NPP_2_normal              0.3739      0.099      3.779      0.000       0.179       0.568
NPP_3_normal              0.3999      0.096      4.157      0.000       0.211       0.589
herb_avg_normal           0.0358      0.026      1.374      0.170      -0.015       0.087
rangeland_acre_normal    -0.9691      0.053    -18.427      0.000      -1.072      -0.866
==============================================================================
Omnibus:                        7.056   Durbin-Watson:                   0.174
Prob(Omnibus):                  0.029   Jarque-Bera (JB):                7.764
Skew:                          -0.199   Prob(JB):                       0.0206
Kurtosis:                       3.468   Cond. No.                         10.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [55]:
X_normal = invent_tall[normal_cols]
X_normal = sm.add_constant(X_normal)
Y = (invent_tall[y_var].astype(float))
ks_normal = sm.OLS(Y, X_normal)
ks_normal_result =ks_normal.fit()
ks_normal_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              inventory   R-squared:                       0.577
Model:                            OLS   Adj. R-squared:                  0.573
Method:                 Least Squares   F-statistic:                     132.9
Date:                Wed, 03 Jan 2024   Prob (F-statistic):           1.27e-88
Time:                        13:00:10   Log-Likelihood:                -7237.8
No. Observations:                 493   AIC:                         1.449e+04
Df Residuals:                     487   BIC:                         1.451e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                  9.928e+05   2.61e+04     38.104      0.000    9.42e+05    1.04e+06
NPP_1_normal           4.257e+05   1.08e+05      3.944      0.000    2.14e+05    6.38e+05
NPP_2_normal           4.624e+05   1.14e+05      4.056      0.000    2.38e+05    6.86e+05
NPP_3_normal           5.422e+05   1.11e+05      4.892      0.000    3.24e+05     7.6e+05
herb_avg_normal       -1.109e+05      3e+04     -3.696      0.000    -1.7e+05    -5.2e+04
rangeland_acre_normal -9.041e+05   6.06e+04    -14.919      0.000   -1.02e+06   -7.85e+05
==============================================================================
Omnibus:                       36.885   Durbin-Watson:                   0.176
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              146.552
Skew:                           0.131   Prob(JB):                     1.50e-32
Kurtosis:                       5.658   Cond. No.                         10.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [57]:
6.38e+05 - 2.14e+05

424000.0